In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
        

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dig = pd.read_csv(r'/kaggle/input/Kannada-MNIST/Dig-MNIST.csv')

In [ ]:
import torch
import random
import torchvision.datasets
import torchvision.transforms
import torchvision.transforms.functional
import matplotlib.pyplot as plt
import copy
import torch.nn.functional as F

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

In [ ]:
seed=116
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

#to reproduce same result over and over
#also we lose some performance in gpu if it set to true
torch.backends.cudnn.deterministic = True

In [ ]:
df = pd.read_csv(r'/kaggle/input/Kannada-MNIST/train.csv')
df = pd.concat([dig,df],axis=0)
df_train = df.iloc[:,:]
df_val = df.iloc[40000:,:]

In [ ]:
labels_train, inputs_train = df_train.iloc[:,0].values,df_train.iloc[:,1:].values
labels_val, inputs_val = df_val.iloc[:,0].values,df_val.iloc[:,1:].values

In [ ]:
class Net(torch.nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution
        # kernel
        self.conv1 = torch.nn.Conv2d(1, 64, 3, padding=1)
        self.conv2 = torch.nn.Conv2d(64, 64, 3, padding=1)
        self.conv3 = torch.nn.Conv2d(64, 128, 3, padding=0)
        self.conv4 = torch.nn.Conv2d(128, 256, 3, padding=0)
        self.conv5 = torch.nn.Conv2d(256, 128, 3, padding=1)
        self.conv6 = torch.nn.Conv2d(128, 128, 3, padding=1)
        self.relu = torch.nn.LeakyReLU()
        self.avgpool1 = torch.nn.AvgPool2d(kernel_size=(5, 5))
        self.fc1 = torch.nn.Linear(128, 10)
        self.maxpool1 = torch.nn.MaxPool2d(2, 2)
        self.maxpool2 = torch.nn.MaxPool2d(2, 2)
        self.dropout = torch.nn.Dropout(0.22)
        # an affine operation: y = Wx + b
        #self.fc1 = torch.nn.Linear(16 * 4 * 4, 120)
        #self.fc2 = torch.nn.Linear(120, 84)
        #self.fc3 = torch.nn.Linear(84, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        
        x = self.conv3(x)
        x = self.relu(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        
        x = self.conv5(x)
        x = self.relu(x)
        x = self.conv6(x)
        x = self.relu(x)
        x = self.avgpool1(x)
        x = x.reshape(x.shape[0], x.shape[1])
        x = self.fc1(x)
    
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [ ]:
def acc_n_loss_func(preds, labels):
    f = torch.nn.CrossEntropyLoss()
  
    loss = f(preds, labels)
    accuracy = (preds.argmax(dim=1) == labels).float().mean()
    return accuracy, loss

In [ ]:
def train_model(model, accnlossfunc, optimizer, scheduler, num_epochs):
    try:
        best_model_wts = copy.deepcopy(model.state_dict())
        best_acc = 0.0
        
        val_accuracy_history = []
        val_loss_history = []
        
        for epoch in range(num_epochs):
            for phase in ['train', 'val']:
                if phase == 'train':
                    dataloader = zip(inputs_train.reshape(-1,7840),labels_train.reshape(-1,10))
                    count_n = len(inputs_train.reshape(-1,7840))
                    model.train()  # Set model to training mode
                else:
                    dataloader = zip(inputs_val.reshape(-1,7840),labels_val.reshape(-1,10))
                    count_n = len(inputs_val.reshape(-1,7840))
                    model.eval()   # Set model to evaluate mode
                running_loss = 0.
                running_acc = 0
                # Iterate over data.
                for i, (inputs, labels) in enumerate(dataloader):
                    
                    inputs = torch.tensor(inputs.reshape(10,1,28,28)).to(device)
                    labels = torch.tensor(labels).type(torch.LongTensor).to(device)
                    labels = labels.flatten()
                    optimizer.zero_grad()
                    # forward and backward
                    with torch.set_grad_enabled(phase == 'train'):
                        
                        
                        preds = model(inputs.float())
                        
                  
           
                        acc, loss_value = accnlossfunc(preds, labels)
                   
                        # backward + optimize only if in training phase
                        if phase == 'train':
                            loss_value.backward()
                            optimizer.step() 
                            scheduler.step()

                    # statistics
                    running_loss += loss_value.item()
                    running_acc += acc.item()
                    print("epoch: {}/{} nested {}/{} {}  - loss: {:.4f} - acc: {:.4f}".format(epoch+1, num_epochs, (i+1), 
                                                            count_n, phase, running_loss/(i+1), running_acc/(i+1)), end='\r')
                epoch_loss = running_loss / count_n
                epoch_acc = running_acc / count_n
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = copy.deepcopy(model.state_dict())
                    val_loss_history.append(epoch_loss)
                    val_accuracy_history.append(epoch_acc)
                print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc), flush=True)
    except (KeyboardInterrupt or RuntimeError) as e:
        model.load_state_dict(best_model_wts)
        print(f"Returning model saved with best accuracy:{best_acc}")
        return model, val_loss_history, val_accuracy_history
        
    model.load_state_dict(best_model_wts)
    print(f"Returning model saved with best accuracy:{best_acc}")
    return model, val_loss_history, val_accuracy_history

In [ ]:
#model = MNISTnet(n_hidden_neurons=64)
model = Net()
model = model.to(device)
loss = acc_n_loss_func
optimizer = torch.optim.Adam(model.parameters(), lr=3e-4)
# Decay LR by a factor of 0.1 every 88 epochs
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2240, gamma=0.7)

In [ ]:
model, val_loss_history, val_accuracy_history = train_model(model, loss, optimizer, scheduler, num_epochs=6)

In [ ]:
test_csv = pd.read_csv(r'/kaggle/input/Kannada-MNIST/test.csv').iloc[:,1:]

test_data = torch.zeros([len(test_csv),1,28,28])

for i in range(len(test_csv)):
    test_data[i,0] = (torch.tensor(test_csv.iloc[i, :]).reshape(28, 28))

In [ ]:
test_preds = model(test_data[:100].to(device))
labels = test_preds.argmax(dim=1)

In [ ]:
batch_size = 1000
result = torch.zeros(len(test_data)).to(device)

for index in range(0, len(test_data), batch_size):
    test_preds = model(test_data[index:index+batch_size].to(device))
    result[index:index+batch_size] = test_preds.argmax(dim=1)
result = result.type(torch.LongTensor)

In [ ]:
submission = pd.read_csv(r'/kaggle/input/Kannada-MNIST/sample_submission.csv')
submission['label'] = result.data.cpu()

submission.to_csv(r'/kaggle/working/submission.csv', index=False)